In [1]:
!pip install langchain
!pip install langchain_community
!pip install langchain-huggingface
!pip install PyPDF2
!pip install transformers
!pip install pathway

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.1 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=c6b17a9b2829779051a3f9128714029e989389bec8a4edb478569330c4e8488f
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a0fd0ef4cfa625679e2658ea1e6fa9c9410558f363daf29fdba189cc30f2c1cd
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing the libraries and intializing them**

In [4]:
import pathway as pw
from pathway.xpacks.llm.vector_store import VectorStoreServer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import PyPDF2
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import pipeline
import tensorflow as tf
from langchain_community.vectorstores import PathwayVectorClient
import re
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


##Function for Extracting text from pdf using pdf_path

In [5]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

In [6]:
def extract_text_from_folder(folder_path):
    pdf_texts = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                pdf_texts.append(extract_text_from_pdf(pdf_path))
    return pdf_texts

In [7]:
Non_publishable_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Non-Publishable")
Unlabeled_papers = extract_text_from_folder("/content/drive/MyDrive/Papers")
CVPR_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Publishable/CVPR")
EMNLP_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Publishable/EMNLP")
KDD_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Publishable/KDD")
NeurIPS_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Publishable/NeurIPS")
TMLR_papers = extract_text_from_folder("/content/drive/MyDrive/Reference/Publishable/TMLR")

In [8]:
Publishable_papers = CVPR_papers + EMNLP_papers + KDD_papers + NeurIPS_papers + TMLR_papers
Labeled_papers = Publishable_papers + Non_publishable_papers
Labeled_papers_labels = [1] * len(Publishable_papers) + [0] * len(Non_publishable_papers)

##Text Preprocessing using Simple NLP Techniques

In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove stop words and non-alphabetical tokens
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Preprocess all texts
Publishable_papers = [preprocess_text(text) for text in Publishable_papers]
Non_publishable_papers = [preprocess_text(text) for text in Non_publishable_papers]
Unlabeled_papers = [preprocess_text(text) for text in Unlabeled_papers]

In [10]:
Labeled_papers = Publishable_papers + Non_publishable_papers

In [11]:
print(Labeled_papers)

['detailed action identification baseball game recording abstract research introduces new complex dataset created nuanced activity recognition baseball video dataset structured support two type analysis one classifying activity segmented video another detecting activity unsegmented continuous video stream study evaluates several method recognizing activity focusing capture temporal organization activity video evaluation start categorizing segmented video progress applying method continuous video feed additionally paper assesses effectiveness different model challenging task forecasting pitch velocity type using baseball broadcast video finding indicate incorporating temporal dynamic model beneficial detailed activity recognition introduction action recognition significant problem computer vision find extensive use sport sional sporting event extensively recorded entertainment recording invaluable subsequent analysis coach scout medium analyst numerous game statistic currently gathered 

Train Test Splitting for Training and Testing the model

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Labeled_papers,Labeled_papers_labels, test_size = 0.20,random_state = 0)

In [13]:
len(X_train)

12

In [14]:
from google.colab import output
output.enable_custom_widget_manager()

##Tokenizing Trainig and Testing data

In [15]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
train_data = tokenizer(
    X_train,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
)

test_data = tokenizer(
    X_test,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
train_data = dict(train_data)

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((train_data, y_train))
batch_size = 8
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

##Training the pretrained model (fine tunning of LLM)

In [18]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(dataset, epochs=5)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
2/2 [==============================] - 27s 329ms/step - loss: 0.6786 - accuracy: 0.5833
Epoch 2/5
2/2 [==============================] - 1s 233ms/step - loss: 0.6132 - accuracy: 0.6667
Epoch 3/5
2/2 [==============================] - 1s 236ms/step - loss: 0.4936 - accuracy: 0.7500
Epoch 4/5
2/2 [==============================] - 1s 233ms/step - loss: 0.3598 - accuracy: 0.9167
Epoch 5/5
2/2 [==============================] - 1s 237ms/step - loss: 0.2082 - accuracy: 1.0000


In [19]:
y_output = model.predict(test_data)

1/1 [==============================] - 2s 2s/step


In [20]:
def get_result(y_output):
  result = []
  for output in y_output['logits']:  # Access the 'logits' key to get the numeric predictions
    logits = tf.constant([output])
    probabilities = tf.nn.softmax(logits, axis=-1)
    # probabilities has shape (1, 2), so access elements using probabilities[0][0] and probabilities[0][1]
    if probabilities[0][0] > probabilities[0][1]:
        result.append(0)
    else:
        result.append(1)
  return result

In [21]:
y_output = get_result(y_output)
print(y_output)

[1, 1, 1]


In [22]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_output)
print("F1 Score:", f1)

F1 Score: 1.0


In [23]:
tokenized_Unlabeled_papers = tokenizer(
    Unlabeled_papers,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
)

In [24]:
y_output = model.predict(tokenized_Unlabeled_papers)
Unlabeled_papers_labels = get_result(y_output)

5/5 [==============================] - 3s 467ms/step


##Using Semi-Supervised Learning for Better Predictions

In [25]:
## Applying Semi-Supervised Learning
Labeled_papers = Labeled_papers + Unlabeled_papers
Labeled_papers_labels = Labeled_papers_labels + Unlabeled_papers_labels

In [26]:
X_train,X_test,y_train,y_test = train_test_split(Labeled_papers,Labeled_papers_labels, test_size = 0.20,random_state = 0)

In [27]:
train_data = tokenizer(
    X_train,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
)

test_data = tokenizer(
    X_test,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
)

In [28]:
train_data = dict(train_data)
dataset = tf.data.Dataset.from_tensor_slices((train_data, y_train))
batch_size = 8
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [29]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(dataset, epochs=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
15/15 [==============================] - 23s 452ms/step - loss: 0.5621 - accuracy: 0.6750
Epoch 2/5
15/15 [==============================] - 7s 451ms/step - loss: 0.1758 - accuracy: 0.9500
Epoch 3/5
15/15 [==============================] - 7s 453ms/step - loss: 0.0960 - accuracy: 0.9833
Epoch 4/5
15/15 [==============================] - 7s 456ms/step - loss: 0.1139 - accuracy: 0.9667
Epoch 5/5
15/15 [==============================] - 7s 462ms/step - loss: 0.0717 - accuracy: 0.9667


In [30]:
from sklearn.metrics import accuracy_score
y_output = model.predict(test_data)
y_output = get_result(y_output)
f1 = f1_score(y_test, y_output)
accuracy = accuracy_score(y_test, y_output)
print("Accuracy:", accuracy)
print("F1 Score:", f1)

1/1 [==============================] - 2s 2s/step
Accuracy: 0.9
F1 Score: 0.9387755102040817


In [31]:
print(y_output)
print(y_test)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]


In [32]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_output)
print("Accuracy:", accuracy)

Accuracy: 0.9


##Function for Predicting is the Paper is Publishable

In [33]:
def is_publishable(paper_text):
  paper_text = preprocess_text(paper_text)
  tokenized_paper_text = tokenizer(
    paper_text,
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='np'  # Return NumPy arrays for compatibility
    )
  y_output = model.predict(tokenized_paper_text)
  y_output = get_result(y_output)
  if y_output[0] == 1:
    return True
  else:
    return False

###Converting .pdf files to .txt file so that pw.io.fs connector can process it.

In [34]:
# Path to the "publishable" folder (source of PDF data)
publishable_folder = '/content/drive/MyDrive/Publishable'

# Path to the "data" folder (destination for text files)
data_folder = '/content/drive/MyDrive/data'

# Create the "data" folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print(f"Created folder: {data_folder}")
else:
    print(f"Folder already exists: {data_folder}")

# Initialize a dictionary to store extracted text
conference_texts = {}

# Iterate through each subfolder in "publishable"
for conference_name in os.listdir(publishable_folder):
    conference_path = os.path.join(publishable_folder, conference_name)

    # Ensure it's a directory
    if os.path.isdir(conference_path):
        conference_texts[conference_name] = []  # Initialize an empty list for this conference

        # Create a folder in "data" for this conference
        conference_data_folder = os.path.join(data_folder, conference_name)
        if not os.path.exists(conference_data_folder):
            os.makedirs(conference_data_folder)
            print(f"Created folder for conference: {conference_name}")

        # Iterate through PDFs in the conference folder
        for pdf_file in os.listdir(conference_path):
            if pdf_file.endswith('.pdf'):
                pdf_path = os.path.join(conference_path, pdf_file)

                # Extract text from the PDF
                pdf_text = ""
                with open(pdf_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    for page in reader.pages:
                        pdf_text += page.extract_text()

                # Add the extracted text to the list for this conference
                conference_texts[conference_name].append(pdf_text)

                # Save the extracted text to a text file in the conference folder
                text_file_name = os.path.splitext(pdf_file)[0] + ".txt"
                text_file_path = os.path.join(conference_data_folder, text_file_name)
                with open(text_file_path, 'w') as text_file:
                    text_file.write(pdf_text)

                print(f"Saved text file: {text_file_path}")

# Final confirmation
print("\nAll PDF texts have been saved into their respective conference folders in 'data'.")

Created folder: /content/drive/MyDrive/data
Created folder for conference: CVPR
Saved text file: /content/drive/MyDrive/data/CVPR/R007.txt
Saved text file: /content/drive/MyDrive/data/CVPR/R006.txt
Saved text file: /content/drive/MyDrive/data/CVPR/R017.txt
Saved text file: /content/drive/MyDrive/data/CVPR/R018.txt
Created folder for conference: NeurIPS
Saved text file: /content/drive/MyDrive/data/NeurIPS/R013.txt
Saved text file: /content/drive/MyDrive/data/NeurIPS/R012.txt
Saved text file: /content/drive/MyDrive/data/NeurIPS/R020.txt
Saved text file: /content/drive/MyDrive/data/NeurIPS/R022.txt
Created folder for conference: TMLR
Saved text file: /content/drive/MyDrive/data/TMLR/R015.txt
Saved text file: /content/drive/MyDrive/data/TMLR/R014.txt
Saved text file: /content/drive/MyDrive/data/TMLR/R023.txt
Saved text file: /content/drive/MyDrive/data/TMLR/R026.txt
Created folder for conference: KDD
Saved text file: /content/drive/MyDrive/data/KDD/R010.txt
Saved text file: /content/drive/

###Using Pathway Connector for Processing the data with streaming mode.

In [35]:
data = pw.io.fs.read(
    '/content/drive/MyDrive/data',
    format="binary",
    mode="streaming",
    with_metadata=True,
)

In [36]:
from google.colab import output
output.enable_custom_widget_manager()

##Initializing and Hosting a Vector Store Server with Caching

In [37]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
splitter = CharacterTextSplitter()

host = "127.0.0.1"
port = 8888

server = VectorStoreServer.from_langchain_components(
    data, embedder=embeddings, splitter=splitter
)
server.run_server(host, port=port, with_cache=True, cache_backend=pw.persistence.Backend.filesystem("./Cache"), threaded=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[str] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


<Thread(VectorStoreServer, started 138821345707584)>

###Connecting to a Pathway Vector Server Using a Client

In [38]:
from langchain_community.vectorstores import PathwayVectorClient

client = PathwayVectorClient(host=host, port=port)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

###PipeLine for processing, extracting_keywords and generate_rationals for the input text

In [40]:
def preprocess_text(text):
    """Preprocesses the text by cleaning it."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text.lower()


In [41]:
def extract_keywords(text, num_keywords=10):
    """Extracts top keywords using TF-IDF."""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    dense = tfidf_matrix.todense()
    tfidf_array = dense[0].tolist()[0]
    sorted_indices = sorted(range(len(tfidf_array)), key=lambda x: tfidf_array[x], reverse=True)
    keywords = [feature_names[i] for i in sorted_indices[:num_keywords]]
    return keywords

In [42]:
def generate_rationale(paper_text, reference_paper_text, conference_name):
    """Generates a rationale explaining why the paper fits the conference."""
    # Preprocess both texts
    paper_text = preprocess_text(paper_text)
    reference_paper_text = preprocess_text(reference_paper_text)

    # Extract key elements
    paper_keywords = extract_keywords(paper_text, num_keywords=70)
    reference_keywords = extract_keywords(reference_paper_text, num_keywords=70)

    # Find shared keywords
    shared_keywords = set(paper_keywords) & set(reference_keywords)

    # Constructing the rationale
    rationale = f"""
    Rationale for Conference Fit (for {conference_name}):

    This paper aligns with the conference's focus due to the shared emphasis on key topics such as {', '.join(shared_keywords)}.
    Furthermore, the findings in {', '.join(shared_keywords)} demonstrate significant contributions to the field of {', '.join(shared_keywords)}, fulfilling the conference's academic standards for groundbreaking work.
    The paper's scope, methodology, and results make it a perfect fit for the conference's theme of {conference_name}.
    """

    # Limit the explanation to 100 words
    rationale = ' '.join(rationale.split()[:100]) + '...'

    return rationale

##Defining Function which finally provide the desired Result

In [43]:
import re

from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel


# Assuming extract_text_from_pdf, is_publishable, and client.similarity_search are defined elsewhere

import os
import re
import logging

logging.basicConfig(level=logging.DEBUG)

def search_conference_with_reason(pdf_path):
    try:
        # Extract text from the given PDF
        given_paper_text = extract_text_from_pdf(pdf_path)
        logging.debug(f"Extracted text from {pdf_path[:100]}")
    except Exception as e:
        return f"Error extracting text from PDF: {e}"

    result = is_publishable(given_paper_text)
    if result:
        try:
            # Perform similarity search to find matching papers
            docs = client.similarity_search(given_paper_text, k=1)
            Document = docs[0]
        except Exception as e:
            return f"Error during similarity search: {e}"

        # Extract path from metadata
        path = Document.metadata.get('path', None)
        if not path:
            return "Path not found or missing in document metadata"

        # Extract conference name
        match = re.search(r"/data/([^/]+)/", path)
        conference_name = match.group(1) if match else "Unknown Conference"
        logging.debug(f"Conference Name: {conference_name}")

        # Read the matched paper
        if not os.path.exists(path):
            return f"File at {path} does not exist"

        try:
            with open(path, 'r') as file:
                matched_paper_text = file.read()
        except Exception as e:
            return f"Error reading file at {path}: {e}"

        # Generate the rationale
        rationale = generate_rationale(given_paper_text, matched_paper_text, conference_name)
        return rationale, conference_name
    else:
        return "This paper is not publishable."


In [44]:
path = "/content/drive/MyDrive/Papers/P017.pdf"
search_conference_with_reason(path)
text = extract_text_from_pdf(path)
result = is_publishable(text)
print(result)

======== Running on http://127.0.0.1:8888 ========
(Press CTRL+C to quit)
1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 70ms/step
True


###Producing derired output for all the 135 pdf files and storing the result in a output.csv file

In [45]:
import csv

def process_papers_in_folder(folder_path, output_csv_path):
    # Open a CSV file for writing the output
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Paper ID", "Publishable", "Conference", "Rationale"])

        # Loop through each PDF in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                paper_id = os.path.splitext(filename)[0]  # Use the filename as Paper ID
                pdf_path = os.path.join(folder_path, filename)

                # Get publishability, conference name, and rationale using search_conference_with_reason
                result = search_conference_with_reason(pdf_path)
                print(f"Debug: Result for {paper_id} - {result}")  # Log the result

                if isinstance(result, tuple) and len(result) == 2:
                    rationale, conference_name = result
                    publishable = "Yes"
                elif isinstance(result, str):
                    rationale = result
                    conference_name = "N/A"
                    publishable = "No"
                else:
                    rationale = "Unknown error"
                    conference_name = "N/A"
                    publishable = "No"

                # Write the row to the CSV file
                writer.writerow([paper_id, publishable, conference_name, rationale])

                # Print a message after processing each paper
                print(f"Processed {paper_id}")

# Example usage
folder_path = "/content/drive/MyDrive/Papers"  # Path to the folder containing the PDFs
output_csv_path = "/content/drive/MyDrive/output.csv"  # Output CSV file path

process_papers_in_folder(folder_path, output_csv_path)


1/1 [==============================] - 0s 66ms/step
Debug: Result for P131 - ("Rationale for Conference Fit (for CVPR): This paper aligns with the conference's focus due to the shared emphasis on key topics such as learning, dataset, training, pretrained, feature, image, maps, approach. Furthermore, the findings in learning, dataset, training, pretrained, feature, image, maps, approach demonstrate significant contributions to the field of learning, dataset, training, pretrained, feature, image, maps, approach, fulfilling the conference's academic standards for groundbreaking work. The paper's scope, methodology, and results make it a perfect fit for the conference's theme of CVPR....", 'CVPR')
Processed P131
1/1 [==============================] - 0s 73ms/step
Debug: Result for P085 - ("Rationale for Conference Fit (for CVPR): This paper aligns with the conference's focus due to the shared emphasis on key topics such as dataset, input, datasets, privacy, protection, training, models, mo

In [46]:
# Instead of using model.save with the 'h5' format:
# model.save('/content/drive/MyDrive/my_model.h5')

# Use the SavedModel format:
model.save('/content/drive/MyDrive/my_model', save_format="tf")

# Alternatively, you can save only the model weights:
model.save_weights('/content/drive/MyDrive/my_model_weights.h5')

###Sorting the rows according to id's from the csv file

In [47]:
import pandas as pd

# Path to your file in Google Drive
file_path = '/content/drive/My Drive/output.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Sort the DataFrame by the 'Paper ID' column
df_sorted = df.sort_values(by='Paper ID')

# Save the sorted DataFrame back to the same file
df_sorted.to_csv(file_path, index=False)

print("The file has been sorted by 'Paper ID' and saved back to the same file.")


The file has been sorted by 'Paper ID' and saved back to the same file.
